In [1]:
import tweepy
import pandas as pd
import csv
import joblib
from lemmatizationText import elaborateText

#import mysql.connector

In [2]:
model = joblib.load("model_lr_count.pkl") #Logistic Regression

def twitter_init():
    # Create variables for each key, secret, token
    consumer_key = "IG3sIHi0ovFvy7ZVygb928qzu"
    consumer_secret = "UXCsm2QYnLTWvliXTq9CPUPgY6oiDFPrWeOFY1u3x6Wv3SX0A5"
    access_token = "1301906875454304263-CQ8EKyDnuxymqez9EWtznKwQXhDZUH"
    access_token_secret = "Yln6i4Mv6PXAmZY8Vz22RGW86PeW6b1y4YZsIEyGDHwcM"

    # Set up OAuth and integrate with API
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True) 

    return api

/home/lorenzo/anaconda3/envs/Ambiente-36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/lorenzo/anaconda3/envs/Ambiente-36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/lorenzo/anaconda3/envs/Ambiente-36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator SelectKBest from version 0.22.2.post1 when using version 0.24.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/lorenzo/anaconda3/envs/Ambiente-36/lib/python3.6/site-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimato

In [20]:
df = pd.DataFrame({
        'Text':["bruto","non saprei","super"],
        'tag':[0,-1,1]
})

print(df)



df = df.replace({'tag':{0:'negative',-1:'neutral',1:'positive'}})

print(df)
print(df.dtypes)


         Text  tag
0       bruto    0
1  non saprei   -1
2       super    1
         Text       tag
0       bruto  negative
1  non saprei   neutral
2       super  positive
Text    object
tag     object
dtype: object
         Text       tag
0       bruto  negative
1  non saprei   neutral
2       super  positive
Text    object
tag     object
dtype: object


In [87]:
list_text = ["love phone","issues phone Apple","fuck bad apple"]
result = model.predict(list_text)
print(result)

list_text = ["love phone"]
result = model.predict(list_text)
print(result)

cleanedText = [elaborateText("Ciao a tutti and got happy for you")]
print(cleanedText)



result = model.predict(cleanedText)
print(result)


[1 0 0]
[1]
['ciao a tutti and get happy for you']
[1]


In [3]:

#Da salvare in file nella versione FINAL
keywords1 ="Apple,@Apple,#Apple,SupportApple"
keywords2 = "battery,Iphone,IPHONE,iPhone,ios,camera,focus,innovation,device,fuck,bad,best,issue,broken,perfect,fantastic,good,love,help"
keywords3 = "Trump,Tesla,Cover,Unboxing,UNBOXING"

keywords_1 = keywords1.split(",")
keywords_2 = keywords2.split(",")
keywords_3 = keywords3.split(",")


#Nella versione finale:
#keywords_1 = keywords_file_1.read().split(",")
#keywords_2 = keywords_file_2.read().split(",")
#keywords_3 = keywords_file_3.read().split(",")

def keywords_filtering(text, keywords_1, keywords_2, keywords_3):
    # keywords filtering as in offline scraping 
    for keyword_1 in keywords_1:
        if keyword_1 in text:
            #for keyword_2 in keywords_2:
             #   if keyword_2 in text:
                    for keyword_3 in keywords_3:
                        if keyword_3 in text:
                            return False
                    return True

In [4]:
tweet_dict = {"index": [],
              "Tweet Datetime": [],
              "Tweet Text": [],
              "tag": [],
              }

tweets_counter = 0
batch_counter = 0
ind = 0


In [7]:
class MyStreamListener(tweepy.StreamListener):

    def on_status(self, status, tweet_mode="extended"):
        #errore subdolo: se non dichiaravo una delle variabili global 
        #uscivo subito dalla funzione!!!!!!!!!!!!!!!!!
        global ind
        global tweets_counter
        global tweet_dict
        global batch_counter
        #print("SONO DENTROOOO")
        # get tweet info 
        id_str = status.id_str
        user_created_at = status.created_at

        try:
            text = status.extended_tweet["full_text"]
        except:
            text = status.text

        
        #username = user.screen_name
        lang = status.lang

        # discard if retweeted tweet
        if "RT" in text or lang != "en":
            return True
        #print("segue il keywords_filtering")
        if keywords_filtering(text, keywords_1, keywords_2, keywords_3):
            
            
            ind+=1
            tweets_counter += 1
            print("{} tweets found...".format(tweets_counter))

            # print tweet info 
            print(text)
            print("Created at: {}".format(user_created_at))

            # add tweet to pandas dataframe
            tweet_dict["index"].append(str(ind))
            print(tweet_dict["index"])
            
            tweet_dict["Tweet Datetime"].append(user_created_at)
            print(tweet_dict["Tweet Datetime"])
                             
            cleanedText = elaborateText(text)
            tweet_dict["Tweet Text"].append(cleanedText)
            print(tweet_dict["Tweet Text"])
            
            #Spero restituisca una lista di tag -1,0,1
            tweet_dict["tag"].append(model.predict([cleanedText]))
                
            cleanedText =[]
            # preprocess and store in DB n-tweets batch
            if tweets_counter == 2:
                batch_counter += 1
                print("Preprocessing and storing into DB tweet batch #{}..." .format(batch_counter))
                
                
                
                
                #save tweets into pandas DataFrame 
                tweet_df = pd.DataFrame(data=tweet_dict)
                
                # Devo tradurre i -1,0,1 in neutral,negative,positive
                tweet_df = tweet_df.replace({'tag':{0:'negative',-1:'neutral',1:'positive'}})
                
                #Devo scrivere sul file csv!!
                # --> quaaaaaaaaaaaaaaaaaa
                #f_writer = open('CleanedTweets.csv','a')
                #writer = csv.writer(f_writer)
                tweet_df.set_index("index")
                tweet_df.to_csv('CleanedTweetsMonitoring.csv',mode='a',index=False,header=False)
                
                


                # reset tweets_counter and dict 
                tweet_dict = {
                      "index": [],
                      "Tweet Datetime": [],
                      "Tweet Text": [],
                      "tag": [],
                      }

                tweets_counter = 0

            return True

In [ ]:
tweet_dict = {"index": [],
              "Tweet Datetime": [],
              "Tweet Text": [],
              "tag": [],
              }

tweets_counter = 0
batch_counter = 0
ind = 0



# creating a stream
api = twitter_init()
myStreamListener = MyStreamListener()
myStream = tweepy.Stream(auth = api.auth, listener=myStreamListener, tweet_mode="extended")
myStream.filter(languages=["en"],track=["Apple"])



1 tweets found...
#brnsunday | #Congress’ proposed #IRA #taxchanges – do they lead to a #secureretirement?; Ready to say goodbye to #passwords? ; Should the new Apple #iPhone be on your #shoppinglist? Market #rotation or market #bust? | @dnlevine, K…https://t.co/sWj1M1UxYW https://t.co/XA6E33nSqS
Created at: 2021-09-19 11:13:40
['1']
[datetime.datetime(2021, 9, 19, 11, 13, 40)]
['brnsunday congress propose ira taxchanges – do they lead to a secureretirement ready to say goodbye to password should the new apple iphone be on your shoppinglist market rotation or market bust k…']
2 tweets found...
Apple suggesting directions to the location it assumes* I’m going to is the Big Tech equivalent of that barista who  kindly asks if you’d like “your usual” but it still rubs you wrong 

*assumes correctly
Created at: 2021-09-19 11:13:40
['1', '2']
[datetime.datetime(2021, 9, 19, 11, 13, 40), datetime.datetime(2021, 9, 19, 11, 13, 40)]
['brnsunday congress propose ira taxchanges – do they lead to 

1 tweets found...
Just saw this on Amazon: 2020 Apple MacBook Air Laptop: Apple M1 Chip, 13” Retina Display, 8GB RAM, 256GB SSD Storage, Backlit Keyboard, FaceTime HD Camera, Touch ID. Works with iPhone/iPad; Gold by Apple for $949.99 https://t.co/Dg9NjTxACN via @amazon
Created at: 2021-09-19 11:14:55
['19']
[datetime.datetime(2021, 9, 19, 11, 14, 55)]
['just saw this on amazon apple macbook air laptop apple m chip retina display gb ram gb ssd storage backlit keyboard facetime hd camera touch id work with iphone ipad gold by apple for via']
2 tweets found...
Sell/buy Apple Watches AirPods And other wearables best price @ https://t.co/XltyaVVjYh  #sgmobmart #sgbuyback #sgipad #ezbuyback https://t.co/8Rnyk3Sp2G
Created at: 2021-09-19 11:15:07
['19', '20']
[datetime.datetime(2021, 9, 19, 11, 14, 55), datetime.datetime(2021, 9, 19, 11, 15, 7)]
['just saw this on amazon apple macbook air laptop apple m chip retina display gb ram gb ssd storage backlit keyboard facetime hd camera touch id wo

1 tweets found...
I just applied Apple cider and I am almost running mad
Created at: 2021-09-19 11:16:04
['35']
[datetime.datetime(2021, 9, 19, 11, 16, 4)]
['i just apply apple cider and i be almost run mad']
2 tweets found...
@nabiplay @Apple Do you look up these jokes that other people wrote?… Please give credit where credit is due
Created at: 2021-09-19 11:16:04
['35', '36']
[datetime.datetime(2021, 9, 19, 11, 16, 4), datetime.datetime(2021, 9, 19, 11, 16, 4)]
['i just apply apple cider and i be almost run mad', 'do you look up these joke that other people write … please give credit where credit be due']
Preprocessing and storing into DB tweet batch #18...
1 tweets found...
@harkl_ It has now been firmly established how the Google/ Apple big tech complex will pander to protect profits.  Projects like yours are despretly needed to make the old guard irrelevant.
Created at: 2021-09-19 11:16:07
['37']
[datetime.datetime(2021, 9, 19, 11, 16, 7)]
['it have now be firmly establish how the

2 tweets found...
Apple’s iOS 15 update arrives Monday, but you might not want to install it right away – CNET https://t.co/S5F1IwIMvq
Created at: 2021-09-19 11:17:27
['55', '56']
[datetime.datetime(2021, 9, 19, 11, 17, 26), datetime.datetime(2021, 9, 19, 11, 17, 27)]
['cette semaine appleevent iphone ipad ipad mini amp apple watch series', 'apple s io update arrives monday but you might not want to install it right away – cnet']
Preprocessing and storing into DB tweet batch #28...
1 tweets found...
Apple iPhone Smartphone of Damian Lewis as Bobby Axelrod in Billions S05E10 TV Show (2021) @Apple #Billions #DamianLewis #BobbyAxelrod #billionsclub #Apple #iPhone #appleiphone #actorsday  https://t.co/IHq6coI9pc
Created at: 2021-09-19 11:17:29
['57']
[datetime.datetime(2021, 9, 19, 11, 17, 29)]
['apple iphone smartphone of damian lewis as bobby axelrod in billion se tv show billion damianlewis bobbyaxelrod billionsclub apple iphone appleiphone actorsday']
2 tweets found...
Bad news, @Apple

2 tweets found...
@tim_cook I have a painful experience of consumer rights protection for more than three years. The fact of infringement is conclusive. So far, Apple has not apologized and compensated me, which makes me very painful! Please @tim_cook contact me, listen and solve my complaints! https://t.co/pVfxTfMABY
Created at: 2021-09-19 11:18:44
['73', '74']
[datetime.datetime(2021, 9, 19, 11, 18, 43), datetime.datetime(2021, 9, 19, 11, 18, 44)]
['have a smirnoff apple with dinner last night next thing i know it s this morning this newfound pandemic zero alcohol tolerance be annoy yo', 'i have a painful experience of consumer right protection for more than three year the fact of infringement be conclusive so far apple have not apologize and compensate me which make me very painful please contact me listen and solve my complaint']
Preprocessing and storing into DB tweet batch #37...
1 tweets found...
@tim_cook I have a painful experience of consumer rights protection for more than t

...

1 tweets found...
🚨 GAMEDAY PODCAST 🚨 

The Dallas Cowboys visit the Los Angeles Chargers today. Who wins?

Check out our official preview show with the Jersey Boyz as @DaveSturchio, @BretErnst, and @americasfan88 get you ready!

Apple: https://t.co/xQMEJXYusW
Spotify: https://t.co/iEoRmeDB4t https://t.co/k9uhnpQSgo
Created at: 2021-09-19 13:00:00
['1923']
[datetime.datetime(2021, 9, 19, 13, 0)]
['gameday podcast the dallas cowboys visit the los angeles charger today who win check out our official preview show with the jersey boyz as and get you ready apple spotify']
2 tweets found...
Ep 25 of Pro Wrestling 4 Life is now live on all podcast platforms, @FiteTV and https://t.co/nXpr07tNph!

Featuring @TheRealXPac @Nick_Hausman @RonFunches &amp; @PCOisNotHuman!

Apple Podcasts: https://t.co/1ERNYmWWUk

Spotify: https://t.co/QSz90Pzeuv https://t.co/CnblpFnTRa
Created at: 2021-09-19 13:00:00
['1923', '1924']
[datetime.datetime(2021, 9, 19, 13, 0), datetime.datetime(2021, 9, 19, 13, 0)]
['ga

1 tweets found...
@Tife_fabunmi @Ogteamie @WondieAFO Check this out while you're here🔥🔥🚀
@TIONWAYNE ft @davido @JAE5_  out now 
Spotify. -https://t.co/5TOG3JOOmz

Apple- 
https://t.co/C7c2nXnI8E

Youtube -https://t.co/2V5k6N1Bgx

Audiomack - https://t.co/XFqdPY8PiV

Boom play - https://t.co/lMknyutqML
Created at: 2021-09-19 13:00:04
['1937']
[datetime.datetime(2021, 9, 19, 13, 0, 4)]
['check this out while you re here ft out now spotify apple youtube audiomack boom play']
2 tweets found...
Apple and Google Remove ‘Navalny’ Voting App in Russia - The New York Times https://t.co/yoVaa9yoZZ
Created at: 2021-09-19 13:00:06
['1937', '1938']
[datetime.datetime(2021, 9, 19, 13, 0, 4), datetime.datetime(2021, 9, 19, 13, 0, 6)]
['check this out while you re here ft out now spotify apple youtube audiomack boom play', 'apple and google remove ‘ navalny voting app in russia the new york time']
Preprocessing and storing into DB tweet batch #969...
1 tweets found...
Apple Folklore &amp; Avalon Stori

1 tweets found...
Apple TV 4K 32GB Black MQD22LL/A - Best Buy https://t.co/1UeFFO3Fk7
Created at: 2021-09-19 13:00:22
['1955']
[datetime.datetime(2021, 9, 19, 13, 0, 22)]
['apple tv k gb black mqdll a best buy']
2 tweets found...
@wentword @leonidvolkov @natynettle I didn't vanish! It was unlawfully removed by Google and Apple
Created at: 2021-09-19 13:00:23
['1955', '1956']
[datetime.datetime(2021, 9, 19, 13, 0, 22), datetime.datetime(2021, 9, 19, 13, 0, 23)]
['apple tv k gb black mqdll a best buy', 'i didn t vanish it be unlawfully remove by google and apple']
Preprocessing and storing into DB tweet batch #978...
1 tweets found...
Season 2 of our podcast MacCast: Unconventional is just around the corner! Make sure to subscribe to be notified of the latest episode. Find it on Apple Podcasts, Spotify, Google Podcasts and Soundcloud: https://t.co/eCH4cnw6vq https://t.co/o0s4vxJVCA
Created at: 2021-09-19 13:00:25
['1957']
[datetime.datetime(2021, 9, 19, 13, 0, 25)]
['season of our podcas

1 tweets found...
PRO TIP *Make sure your content and donation pages are mobile-friendly! Use text-to-donate options and mobile-friendly systems like Apple and Google Pay to accept donations* Join us at the Summit. Registration is now LIVE. https://t.co/YcVbAfn0O8 https://t.co/3lJ7SHlTCl
Created at: 2021-09-19 13:00:46
['1971']
[datetime.datetime(2021, 9, 19, 13, 0, 46)]
['pro tip make sure your content and donation page be mobile friendly use text to donate option and mobile friendly system like apple and google pay to accept donation join us at the summit registration be now live']
2 tweets found...
Apple Music is the last Apple product I use and only because I didn't want to recreate my entire library...and they keep fucking up. I'm about to have to kick em to the curb.
Created at: 2021-09-19 13:00:46
['1971', '1972']
[datetime.datetime(2021, 9, 19, 13, 0, 46), datetime.datetime(2021, 9, 19, 13, 0, 46)]
['pro tip make sure your content and donation page be mobile friendly use text t

2 tweets found...
#gamechanger device arrived 🎉
Damson &amp; Apple crumble for tea 😊 https://t.co/Ui8VRqK3Yb
Created at: 2021-09-19 13:01:39
['1987', '1988']
[datetime.datetime(2021, 9, 19, 13, 1, 36), datetime.datetime(2021, 9, 19, 13, 1, 39)]
['i be not who you think i be pre order today indie apple bn amzn', 'gamechanger device arrive damson amp apple crumble for tea']
Preprocessing and storing into DB tweet batch #994...
1 tweets found...
@Super70sSports My Dad used to go to McDs back then and just get get coffee in the little styrofoam cup. Same temp as the Apple Pie and still steaming three hours later.
Created at: 2021-09-19 13:01:39
['1989']
[datetime.datetime(2021, 9, 19, 13, 1, 39)]
['my dad use to go to mcds back then and just get get coffee in the little styrofoam cup same temp as the apple pie and still steam three hour late']
2 tweets found...
THE CIRCLE VIEW is a rock band based in Chicago. You can find them on Apple Music and Spotify or any other streaming service. They